In [1]:
import os, glob

import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme.expressions import Beta, DefineVariable
from biogeme.models import loglogit, piecewiseFormula
from biogeme.models import lognested
from biogeme.models import logcnl_avail

pandas = pd.read_table("lpmc02.dat")
database = db.Database("LPMC",pandas)
pd.options.display.float_format = '{:.3g}'.format

globals().update(database.variables)

# Exclude
# exclude = (  ArrivalTimeHours_1   ==  -1  )
# database.remove(exclude)
  
# Choice
chosenAlternative = travel_mode


#Parameters to be estimated+ (  BestAlternative_4   *  4  )
# Arguments:
#   1  Name for report. Typically, the same as the variable
#   2  Starting value
#   3  Lower bound
#   4  Upper bound
#   5  0: estimate the parameter, 1: keep it fixed
Constant1 = Beta('Constant1',0,None,None,1)
Constant2 = Beta('Constant2',0,None,None,0)
Constant3 = Beta('Constant3',0,None,None,0)
Constant4 = Beta('Constant4',0,None,None,0)
Cost = Beta('Cost',0,None,None,0)
Total_TT1 = Beta('Total_TT1',0,None,None,0)
Total_TT2 = Beta('Total_TT2',0,None,None,0)
Total_TT3 = Beta('Total_TT3',0,None,None,0)
Total_TT4 = Beta('Total_TT4',0,None,None,0)

CarOwn_2 = Beta('CarOwn_2',0,None,None,0)
CarOwn_3 = Beta('CarOwn_3',0,None,None,0)
CarOwn_4 = Beta('CarOwn_4',0,None,None,0)

LAMBDA = Beta('LAMBDA',1,None,None,0)

# parameters relevant to the nests
N_SM = Beta('N_SM',1,1,None, 0)
N_MOTOR = Beta('N_MOTOR',1,1,None, 0)

a_SM_WALK = Beta('a_SM_WALK',1,0,1,1)
a_SM_CYC = Beta('a_SM_CYC',0.5,0,1,0)
a_MOTOR_PT = Beta('a_MOTOR_PT',1,0,1,1)
a_MOTOR_CAR = Beta('a_MOTOR_CAR',1,0,1,1)


# socio-economic factors (interacting with Time)
Time_Age_1 = Beta('Time_Age_1', 0, None, None, 0)
Time_Age_2 = Beta('Time_Age_2', 0, None, None, 0)
Time_Age_3 = Beta('Time_Age_3', 0, None, None, 0)
Time_Age_4 = Beta('Time_Age_4', 0, None, None, 0)


# Utilities

#Opt1 = walking
#Opt2 = cycling
#Opt3 = public transport
#Opt4 = driving


cost_public = DefineVariable('cost_public', cost_transit ,database)
dur_public = DefineVariable('dur_public', (dur_pt_access + dur_pt_rail + dur_pt_bus + dur_pt_int),database)
cost_driving = DefineVariable('cost_driving', cost_driving_fuel + cost_driving_ccharge ,database)

Opt1 = Constant1 + Total_TT1 * ((dur_walking) ** LAMBDA -1)/LAMBDA + Time_Age_1 * dur_walking * age
Opt2 = Constant2 + Total_TT2 * ((dur_cycling) ** LAMBDA -1)/LAMBDA+ CarOwn_2 * car_ownership +\
                    Time_Age_2 * dur_cycling * age
Opt3 = Constant3 + Cost * cost_public + Total_TT3 * (dur_public ** LAMBDA -1)/LAMBDA + CarOwn_3 * car_ownership +\
                    Time_Age_3 * dur_public * age
Opt4 = Constant4 + Cost * cost_driving + Total_TT4 * ((dur_driving) ** LAMBDA -1)/LAMBDA +\
                    CarOwn_4 * car_ownership + Time_Age_4 * dur_driving * age


V = {1: Opt1,2: Opt2,3: Opt3,4: Opt4}
av = {1: 1,2: 1,3: 1,4: 1}


#Definitions of nests
alpha_N_SM = {1: a_SM_WALK, 2: a_SM_CYC, 3: 0, 4: 0}
alpha_N_MOTOR = {1: 0, 2: 1 - a_SM_CYC, 3: a_MOTOR_PT, 4: a_MOTOR_CAR}

nest_N_SM = N_SM, alpha_N_SM
nest_N_MOTOR = N_MOTOR, alpha_N_MOTOR

nests = nest_N_SM, nest_N_MOTOR


output_dir = "./model-nested-output"
filepath = os.path.join(output_dir, "logit_CN_lpmc_sm_motor_bike")
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
    
# delete previously saved html and pickle
for file in glob.glob(f"{filepath}*"):
    os.remove(file)
    
# The choice model is a logit, with availability conditions
logprob = logcnl_avail(V, av, nests, chosenAlternative)
biogeme  = bio.BIOGEME(database,logprob)
biogeme.modelName = filepath
results = biogeme.estimate()

# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(f"Nbr of observations: {database.getNumberOfObservations()}")
print(f"LL(0) =    {results.data.initLogLike:.3f}")
print(f"LL(beta) = {results.data.logLike:.3f}")
print(f"rho bar square = {results.data.rhoBarSquare:.3g}")
print(f"Output file: {results.data.htmlFileName}")

pandasResults



Nbr of observations: 5000
LL(0) =    -6931.472
LL(beta) = -3650.218
rho bar square = 0.471
Output file: ./model-nested-output/logit_CN_lpmc_sm_motor_bike.html


,Value,Active bound,Std err,t-test,p-value,Rob. Std err,Rob. t-test,Rob. p-value
CarOwn_2,-0.199,0,0.138,-1.44,0.151,0.148,-1.35,0.178
CarOwn_3,-0.24,0,0.138,-1.74,0.0824,0.16,-1.5,0.135
CarOwn_4,0.819,0,0.0901,9.08,0,0.0977,8.38,0
Constant2,-2.31,0,11.4,-0.203,0.839,0.43,-5.37,7.96e-08
Constant3,2,0,0.235,8.54,0,0.228,8.81,0
Constant4,0.0384,0,0.398,0.0964,0.923,0.465,0.0825,0.934
Cost,-0.0917,0,0.0198,-4.62,3.83e-06,0.0247,-3.71,0.00021
LAMBDA,-0.00908,0,0.102,-0.0892,0.929,0.121,-0.0753,0.94
N_MOTOR,1.64,0,0.279,5.87,4.28e-09,0.338,4.86,1.17e-06
N_SM,1,1,0.221,4.53,5.92e-06,0.307,3.25,0.00115
